In [ ]:
import numpy as np

def dB_to_lin(pow_dB):
    return 10**(np.array(pow_dB)/10)

def lin_to_dB(pow_lin):
    return 10*np.log10(np.array(pow_lin))

rssi_3470_3510 = np.array([-85.53,
-96.04,
-96.62,
-102.69,
-95.77,
-100.64,
-94.44,
-101.19,
-97.66,
-88.8
])


rssi_3610_3650 = np.array([-84.93,
-98.94,
-102.76,
-103.86,
-94.13,
-103.55,
-86.8,
-99.54,
-103.67,
-89.61
])


rssi_2504_2544 = np.array([-92.7,
-77.71,
-77.51,
-90.83,
-90.13,
-78.96,
-84.21,
-81.1,
-85.72,
-77.35
])


rssi_5190_5210 = np.array([-106.64,
-105.2,
-106.97,
-101.91,
-105.03,
-103.33,
-101.26,
-105.88,
-107.03,
-105.28
])

In [ ]:
dc_3470_3510 = np.array([45.25,
99.88,
99.49,
52.85,
54.27,
97.45,
65.29,
95.48,
76.99,
100
])


dc_3610_3650 = np.array([45.25,
99.59,
98.16,
43.26,
52.47,
92.28,
44.88,
99.03,
57.14,
100
])


dc_2504_2544 = np.array([99.96,
100,
100,
100,
100,
100,
100,
100,
100,
100
])

In [ ]:
import os
import scipy.io as sio
import matplotlib.pyplot as plt
map_folderdir = "./"
directory = os.listdir(map_folderdir)
flag = 0
for fname in directory:
    if "SLCmap" in fname:
        map_file = os.path.join(map_folderdir, fname)
        flag = 1

if flag == 0:
    errorMessage = 'Error: The file does not exist in the folder:\n ' + map_folderdir
    warnings.warn(errorMessage)

print('Now reading ' + map_file + "\n")
x = sio.loadmat(map_file)
map_struct = x['SLC']

# Define a new struct named SLC
SLC = map_struct[0][0]
column_map = dict(zip([name for name in SLC.dtype.names], [i for i in range(len(SLC.dtype.names))]))

map_ = SLC[column_map["dem"]] + 0.3048 * SLC[column_map["hybrid_bldg"]]
map_ = map_[::10, ::10]

### Implement Pseudo-Inverse

In [ ]:
data_points = np.array([
    [966, 2992],
    [2569, 3767],
    [2873, 3447],
    [2621, 4286],
    [1312, 3830],
    [3828, 2667],
    [242, 2442],
    [1711, 3145],
    [2852, 1584],
    [1903, 2393]
])
data_points = data_points.astype(float)/10.0
data_points = np.floor(data_points).astype("int")

In [ ]:
def serialize_index(row, col, num_cols=map_.shape[1]):
    return row * num_cols + col

dp_serial = []
for dp in data_points:
    serial_dp = serialize_index(dp[1], dp[0])
    dp_serial.append(serial_dp)
    
dp_serial = np.array(dp_serial)
dp_serial

In [ ]:
from tqdm import tqdm
def deserialize_index(serialized_index, num_cols=map_.shape[1]):
    row = serialized_index // num_cols
    col = serialized_index % num_cols
    return row, col

def euclidean_distance(serialized_index1, serialized_index2, num_cols=map_.shape[1]):
    row1, col1 = deserialize_index(serialized_index1, num_cols)
    row2, col2 = deserialize_index(serialized_index2, num_cols)

    distance = ((row2 - row1)**2 + (col2 - col1)**2)**0.5*(0.5*10)
    if distance<1:
        distance=1
    return distance

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from tqdm import tqdm
import matplotlib
matplotlib.rcParams['font.family'] = 'Times New Roman'


def lin_to_dB(pow_lin):
    return 10*np.log10(np.array(pow_lin))


def aggregate_data(data, block_size):
    """
    Aggregates the data into larger blocks by averaging.

    Parameters:
    - data (np.array): 2D array of the data.
    - block_size (int): The size of the block to aggregate the data into.

    Returns:
    - np.array: Aggregated data.
    """
    # Define the new shape based on the block size
    shape = (data.shape[0] // block_size, data.shape[1] // block_size)
    aggregated_data = np.zeros(shape)

    for i in range(shape[0]):
        for j in range(shape[1]):
            aggregated_data[i, j] = np.mean(data[i*block_size:(i+1)*block_size, 
                                                 j*block_size:(j+1)*block_size])
    return aggregated_data

def draw_aggregated_squares(ax, power_data, duty_cycle_data, block_size):
    """
    Draws aggregated squares inside each pixel based on the aggregated duty cycle and power data.

    Parameters:
    - ax: The matplotlib axis to draw on.
    - power_data: 2D numpy array with the average power values.
    - duty_cycle_data: 2D numpy array with the duty cycle values.
    - block_size: The size of each block of aggregated data.
    """
    # Define the colormap for the average power
    power_cmap = plt.cm.jet

    # Normalize the power data for the colormap
    power_norm = mcolors.Normalize(vmin=np.min(power_data), vmax=np.max(power_data))

    # Calculate the side length of each aggregated block
    pixel_size = block_size

    # Iterate over the grid of aggregated data
    for i in tqdm(range(power_data.shape[0])):
        for j in range(power_data.shape[1]):
            # Calculate the size of the square based on the duty cycle
            square_side_length = (duty_cycle_data[i, j])**(1/1.1) * pixel_size

            # Calculate the color based on the average power
            square_color = power_cmap(power_norm(power_data[i, j]))

            # Calculate the coordinates of the bottom-left corner of the square
            square_x = j * pixel_size + (pixel_size - square_side_length) / 2
            square_y = i * pixel_size + (pixel_size - square_side_length) / 2

            # Draw the square for the duty cycle
            ax.add_patch(plt.Rectangle((square_x, square_y), square_side_length, square_side_length, 
                                       color=square_color))

            # Draw the white background for the block
            ax.add_patch(plt.Rectangle((j * pixel_size, i * pixel_size), pixel_size, pixel_size, 
                                       edgecolor='grey', facecolor='white', zorder=0))

# Example usage
fig, ax = plt.subplots(figsize=(10, 8))
ax.set_aspect('equal')

# Create some example data
power_data = lin_to_dB(np.load("signal_estimates_3470_3510_low_res_cov.npy")).reshape(map_.shape)
print(power_data.shape)
duty_cycle_data = np.load("duty_cycle_3470_3510_idw.npy")/100
# The block size for aggregation
block_size = 14

# Aggregate the data
aggregated_power = aggregate_data(power_data, block_size)
aggregated_duty_cycle = aggregate_data(duty_cycle_data, block_size)

# Draw the aggregated squares
draw_aggregated_squares(ax, aggregated_power, aggregated_duty_cycle, block_size)

# Set the limits and turn off the axes
plt.xlim(0, aggregated_power.shape[1] * block_size)
plt.ylim(0, aggregated_power.shape[0] * block_size)
#plt.axis('off')

# Create a ScalarMappable with your colormap and normalization
sm = plt.cm.ScalarMappable(cmap=plt.cm.jet, norm=plt.Normalize(vmin=np.min(power_data), vmax=np.max(power_data)))

# Create the colorbar with a smaller size
cbar = plt.colorbar(sm, ax=ax, shrink=0.8, pad=0.02)  # adjust the shrink value as needed

# Set a larger font size for the colorbar label
cbar.set_label('Average Power [dBX]', size=20)  # adjust the size value as needed

# Increase font size of colorbar ticks
cbar.ax.tick_params(labelsize=19)
# Other plot settings...

# Show the plot
plt.tight_layout()
plt.title("3470-3510 MHz Spectrum Occupancy", fontsize=27)
N1 = map_.shape[0]
N2 = map_.shape[1]

en = SLC[column_map["axis"]]
UTM_long = np.linspace(en[0, 2], en[0, 3] - SLC[column_map["cellsize"]], N1)
UTM_lat = np.linspace(en[0, 0], en[0, 1] - SLC[column_map["cellsize"]], N2)
UTM_long = np.squeeze(UTM_long)
UTM_lat = np.squeeze(UTM_lat)
interval = 100
ax.set_xticks(range(0, len(UTM_lat), interval))
ax.set_xticklabels(UTM_lat[::interval], fontsize=14, rotation=0)

ax.set_yticks(range(0, len(UTM_long), interval))
ax.set_yticklabels(UTM_long[::interval], fontsize=14)



interval = max(1, len(UTM_lat) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_lat), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_lat[::interval]]

# Set the tick values and labels
plt.xticks(ticks=tick_values, labels=tick_labels, fontsize=14, rotation=0)

interval = max(1, len(UTM_long) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_long), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_long[::interval]]

# Adjust the overlapping xticks and yticks by setting the first ytick to be empty
plt.yticks(ticks=[0] + tick_values[1:], labels=[""] + tick_labels[1:], fontsize=14, rotation=90)



# Setting X-axis title
ax.set_xlabel('UTM$_E$ [m]', fontsize=24)
ax.set_ylabel('UTM$_N$ [m]', fontsize=24)
ax.xaxis.labelpad = 10  # title standoff
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from tqdm import tqdm
import matplotlib

matplotlib.rcParams['font.family'] = 'Times New Roman'

def lin_to_dB(pow_lin):
    return 10*np.log10(np.array(pow_lin))


def aggregate_data(data, block_size):
    """
    Aggregates the data into larger blocks by averaging.

    Parameters:
    - data (np.array): 2D array of the data.
    - block_size (int): The size of the block to aggregate the data into.

    Returns:
    - np.array: Aggregated data.
    """
    # Define the new shape based on the block size
    shape = (data.shape[0] // block_size, data.shape[1] // block_size)
    aggregated_data = np.zeros(shape)

    for i in range(shape[0]):
        for j in range(shape[1]):
            aggregated_data[i, j] = np.mean(data[i*block_size:(i+1)*block_size, 
                                                 j*block_size:(j+1)*block_size])
    return aggregated_data

def draw_aggregated_squares(ax, power_data, duty_cycle_data, block_size):
    """
    Draws aggregated squares inside each pixel based on the aggregated duty cycle and power data.

    Parameters:
    - ax: The matplotlib axis to draw on.
    - power_data: 2D numpy array with the average power values.
    - duty_cycle_data: 2D numpy array with the duty cycle values.
    - block_size: The size of each block of aggregated data.
    """
    # Define the colormap for the average power
    power_cmap = plt.cm.jet

    # Normalize the power data for the colormap
    power_norm = mcolors.Normalize(vmin=np.min(power_data), vmax=np.max(power_data))

    # Calculate the side length of each aggregated block
    pixel_size = block_size

    # Iterate over the grid of aggregated data
    for i in tqdm(range(power_data.shape[0])):
        for j in range(power_data.shape[1]):
            # Calculate the size of the square based on the duty cycle
            square_side_length = (duty_cycle_data[i, j])**(1/1.1) * pixel_size

            # Calculate the color based on the average power
            square_color = power_cmap(power_norm(power_data[i, j]))

            # Calculate the coordinates of the bottom-left corner of the square
            square_x = j * pixel_size + (pixel_size - square_side_length) / 2
            square_y = i * pixel_size + (pixel_size - square_side_length) / 2

            # Draw the square for the duty cycle
            ax.add_patch(plt.Rectangle((square_x, square_y), square_side_length, square_side_length, 
                                       color=square_color))

# Example usage
fig, ax = plt.subplots(figsize=(10, 8))
ax.set_aspect('equal')

# Create some example data
power_data = lin_to_dB(np.load("signal_estimates_2160_2170_low_res_cov2.npy")).reshape(map_.shape)
print(power_data.shape)
duty_cycle_data = np.load("duty_cycle_2160_2170_idw2.npy")/100
# The block size for aggregation
block_size = 14

# Aggregate the data
aggregated_power = aggregate_data(power_data, block_size)
aggregated_duty_cycle = aggregate_data(duty_cycle_data, block_size)

# Draw the aggregated squares
draw_aggregated_squares(ax, aggregated_power, aggregated_duty_cycle, block_size)

# Set the limits and turn off the axes
plt.xlim(0, aggregated_power.shape[1] * block_size)
plt.ylim(0, aggregated_power.shape[0] * block_size)
#plt.axis('off')

# Create a ScalarMappable with your colormap and normalization
sm = plt.cm.ScalarMappable(cmap=plt.cm.jet, norm=plt.Normalize(vmin=np.min(power_data), vmax=np.max(power_data)))

# Create the colorbar with a smaller size
cbar = plt.colorbar(sm, ax=ax, shrink=0.8, pad=0.02)  # adjust the shrink value as needed

# Set a larger font size for the colorbar label
cbar.set_label('Average Power [dBX]', size=20)  # adjust the size value as needed

# Increase font size of colorbar ticks
cbar.ax.tick_params(labelsize=19)
# Other plot settings...

# Show the plot
plt.tight_layout()
plt.title("2160.5-2169.5 MHz Spectrum Occupancy", fontsize=27)
N1 = map_.shape[0]
N2 = map_.shape[1]

en = SLC[column_map["axis"]]
UTM_long = np.linspace(en[0, 2], en[0, 3] - SLC[column_map["cellsize"]], N1)
UTM_lat = np.linspace(en[0, 0], en[0, 1] - SLC[column_map["cellsize"]], N2)
UTM_long = np.squeeze(UTM_long)
UTM_lat = np.squeeze(UTM_lat)
interval = 100
ax.set_xticks(range(0, len(UTM_lat), interval))
ax.set_xticklabels(UTM_lat[::interval], fontsize=14, rotation=0)

ax.set_yticks(range(0, len(UTM_long), interval))
ax.set_yticklabels(UTM_long[::interval], fontsize=14)



interval = max(1, len(UTM_lat) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_lat), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_lat[::interval]]

# Set the tick values and labels
plt.xticks(ticks=tick_values, labels=tick_labels, fontsize=14, rotation=0)

interval = max(1, len(UTM_long) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_long), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_long[::interval]]

# Adjust the overlapping xticks and yticks by setting the first ytick to be empty
plt.yticks(ticks=[0] + tick_values[1:], labels=[""] + tick_labels[1:], fontsize=14, rotation=90)



# Setting X-axis title
ax.set_xlabel('UTM$_E$ [m]', fontsize=24)
ax.set_ylabel('UTM$_N$ [m]', fontsize=24)
ax.xaxis.labelpad = 10  # title standoff
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from tqdm import tqdm
import matplotlib

matplotlib.rcParams['font.family'] = 'Times New Roman'

def lin_to_dB(pow_lin):
    return 10*np.log10(np.array(pow_lin))


def aggregate_data(data, block_size):
    """
    Aggregates the data into larger blocks by averaging.

    Parameters:
    - data (np.array): 2D array of the data.
    - block_size (int): The size of the block to aggregate the data into.

    Returns:
    - np.array: Aggregated data.
    """
    # Define the new shape based on the block size
    shape = (data.shape[0] // block_size, data.shape[1] // block_size)
    aggregated_data = np.zeros(shape)

    for i in range(shape[0]):
        for j in range(shape[1]):
            aggregated_data[i, j] = np.mean(data[i*block_size:(i+1)*block_size, 
                                                 j*block_size:(j+1)*block_size])
    return aggregated_data

def draw_aggregated_squares(ax, power_data, duty_cycle_data, block_size):
    """
    Draws aggregated squares inside each pixel based on the aggregated duty cycle and power data.

    Parameters:
    - ax: The matplotlib axis to draw on.
    - power_data: 2D numpy array with the average power values.
    - duty_cycle_data: 2D numpy array with the duty cycle values.
    - block_size: The size of each block of aggregated data.
    """
    # Define the colormap for the average power
    power_cmap = plt.cm.jet

    # Normalize the power data for the colormap
    power_norm = mcolors.Normalize(vmin=np.min(power_data), vmax=np.max(power_data))

    # Calculate the side length of each aggregated block
    pixel_size = block_size

    # Iterate over the grid of aggregated data
    for i in tqdm(range(power_data.shape[0])):
        for j in range(power_data.shape[1]):
            # Calculate the size of the square based on the duty cycle
            square_side_length = (duty_cycle_data[i, j])**(1/1.1) * pixel_size

            # Calculate the color based on the average power
            square_color = power_cmap(power_norm(power_data[i, j]))

            # Calculate the coordinates of the bottom-left corner of the square
            square_x = j * pixel_size + (pixel_size - square_side_length) / 2
            square_y = i * pixel_size + (pixel_size - square_side_length) / 2

            # Draw the square for the duty cycle
            ax.add_patch(plt.Rectangle((square_x, square_y), square_side_length, square_side_length, 
                                       color=square_color))

# Example usage
fig, ax = plt.subplots(figsize=(10, 8))
ax.set_aspect('equal')

# Create some example data
power_data = lin_to_dB(np.load("signal_estimates_3610_3650_low_res_cov3.npy")).reshape(map_.shape)
print(power_data.shape)
duty_cycle_data = np.load("duty_cycle_3610_3650_idw2.npy")/100
# The block size for aggregation
block_size = 14

# Aggregate the data
aggregated_power = aggregate_data(power_data, block_size)
aggregated_duty_cycle = aggregate_data(duty_cycle_data, block_size)

# Draw the aggregated squares
draw_aggregated_squares(ax, aggregated_power, aggregated_duty_cycle, block_size)

# Set the limits and turn off the axes
plt.xlim(0, aggregated_power.shape[1] * block_size)
plt.ylim(0, aggregated_power.shape[0] * block_size)
#plt.axis('off')

# Create a ScalarMappable with your colormap and normalization
sm = plt.cm.ScalarMappable(cmap=plt.cm.jet, norm=plt.Normalize(vmin=np.min(power_data), vmax=np.max(power_data)))

# Create the colorbar with a smaller size
cbar = plt.colorbar(sm, ax=ax, shrink=0.8, pad=0.02)  # adjust the shrink value as needed

# Set a larger font size for the colorbar label
cbar.set_label('Average Power [dBX]', size=20)  # adjust the size value as needed

# Increase font size of colorbar ticks
cbar.ax.tick_params(labelsize=19)
# Other plot settings...

# Show the plot
plt.tight_layout()
plt.title("3610-3650 MHz Spectrum Occupancy", fontsize=27)
N1 = map_.shape[0]
N2 = map_.shape[1]

en = SLC[column_map["axis"]]
UTM_long = np.linspace(en[0, 2], en[0, 3] - SLC[column_map["cellsize"]], N1)
UTM_lat = np.linspace(en[0, 0], en[0, 1] - SLC[column_map["cellsize"]], N2)
UTM_long = np.squeeze(UTM_long)
UTM_lat = np.squeeze(UTM_lat)
interval = 100
ax.set_xticks(range(0, len(UTM_lat), interval))
ax.set_xticklabels(UTM_lat[::interval], fontsize=14, rotation=0)

ax.set_yticks(range(0, len(UTM_long), interval))
ax.set_yticklabels(UTM_long[::interval], fontsize=14)



interval = max(1, len(UTM_lat) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_lat), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_lat[::interval]]

# Set the tick values and labels
plt.xticks(ticks=tick_values, labels=tick_labels, fontsize=14, rotation=0)

interval = max(1, len(UTM_long) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_long), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_long[::interval]]

# Adjust the overlapping xticks and yticks by setting the first ytick to be empty
plt.yticks(ticks=[0] + tick_values[1:], labels=[""] + tick_labels[1:], fontsize=14, rotation=90)



# Setting X-axis title
ax.set_xlabel('UTM$_E$ [m]', fontsize=24)
ax.set_ylabel('UTM$_N$ [m]', fontsize=24)
ax.xaxis.labelpad = 10  # title standoff
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from tqdm import tqdm
import matplotlib

matplotlib.rcParams['font.family'] = 'Times New Roman'

def lin_to_dB(pow_lin):
    return 10*np.log10(np.array(pow_lin))


def aggregate_data(data, block_size):
    """
    Aggregates the data into larger blocks by averaging.

    Parameters:
    - data (np.array): 2D array of the data.
    - block_size (int): The size of the block to aggregate the data into.

    Returns:
    - np.array: Aggregated data.
    """
    # Define the new shape based on the block size
    shape = (data.shape[0] // block_size, data.shape[1] // block_size)
    aggregated_data = np.zeros(shape)

    for i in range(shape[0]):
        for j in range(shape[1]):
            aggregated_data[i, j] = np.mean(data[i*block_size:(i+1)*block_size, 
                                                 j*block_size:(j+1)*block_size])
    return aggregated_data

def draw_aggregated_squares(ax, power_data, duty_cycle_data, block_size):
    """
    Draws aggregated squares inside each pixel based on the aggregated duty cycle and power data.

    Parameters:
    - ax: The matplotlib axis to draw on.
    - power_data: 2D numpy array with the average power values.
    - duty_cycle_data: 2D numpy array with the duty cycle values.
    - block_size: The size of each block of aggregated data.
    """
    # Define the colormap for the average power
    power_cmap = plt.cm.jet

    # Normalize the power data for the colormap
    power_norm = mcolors.Normalize(vmin=np.min(power_data), vmax=np.max(power_data))

    # Calculate the side length of each aggregated block
    pixel_size = block_size

    # Iterate over the grid of aggregated data
    for i in tqdm(range(power_data.shape[0])):
        for j in range(power_data.shape[1]):
            # Calculate the size of the square based on the duty cycle
            square_side_length = (duty_cycle_data[i, j])**(1/1.1) * pixel_size

            # Calculate the color based on the average power
            square_color = power_cmap(power_norm(power_data[i, j]))

            # Calculate the coordinates of the bottom-left corner of the square
            square_x = j * pixel_size + (pixel_size - square_side_length) / 2
            square_y = i * pixel_size + (pixel_size - square_side_length) / 2

            # Draw the square for the duty cycle
            ax.add_patch(plt.Rectangle((square_x, square_y), square_side_length, square_side_length, 
                                       color=square_color))

# Example usage
fig, ax = plt.subplots(figsize=(10, 8))
ax.set_aspect('equal')

# Create some example data
power_data = lin_to_dB(np.load("signal_estimates_3470_3510_low_res_cov3.npy")).reshape(map_.shape)
print(power_data.shape)
duty_cycle_data = np.load("duty_cycle_3470_3510_idw2.npy")/100
# The block size for aggregation
block_size = 14

# Aggregate the data
aggregated_power = aggregate_data(power_data, block_size)
aggregated_duty_cycle = aggregate_data(duty_cycle_data, block_size)

# Draw the aggregated squares
draw_aggregated_squares(ax, aggregated_power, aggregated_duty_cycle, block_size)

# Set the limits and turn off the axes
plt.xlim(0, aggregated_power.shape[1] * block_size)
plt.ylim(0, aggregated_power.shape[0] * block_size)
#plt.axis('off')

# Create a ScalarMappable with your colormap and normalization
sm = plt.cm.ScalarMappable(cmap=plt.cm.jet, norm=plt.Normalize(vmin=np.min(power_data), vmax=np.max(power_data)))

# Create the colorbar with a smaller size
cbar = plt.colorbar(sm, ax=ax, shrink=0.8, pad=0.02)  # adjust the shrink value as needed

# Set a larger font size for the colorbar label
cbar.set_label('Average Power [dBX]', size=20)  # adjust the size value as needed

# Increase font size of colorbar ticks
cbar.ax.tick_params(labelsize=19)
# Other plot settings...

# Show the plot
plt.tight_layout()
plt.title("3470-3510 MHz Spectrum Occupancy", fontsize=27)
N1 = map_.shape[0]
N2 = map_.shape[1]

en = SLC[column_map["axis"]]
UTM_long = np.linspace(en[0, 2], en[0, 3] - SLC[column_map["cellsize"]], N1)
UTM_lat = np.linspace(en[0, 0], en[0, 1] - SLC[column_map["cellsize"]], N2)
UTM_long = np.squeeze(UTM_long)
UTM_lat = np.squeeze(UTM_lat)
interval = 100
ax.set_xticks(range(0, len(UTM_lat), interval))
ax.set_xticklabels(UTM_lat[::interval], fontsize=14, rotation=0)

ax.set_yticks(range(0, len(UTM_long), interval))
ax.set_yticklabels(UTM_long[::interval], fontsize=14)



interval = max(1, len(UTM_lat) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_lat), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_lat[::interval]]

# Set the tick values and labels
plt.xticks(ticks=tick_values, labels=tick_labels, fontsize=14, rotation=0)

interval = max(1, len(UTM_long) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_long), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_long[::interval]]

# Adjust the overlapping xticks and yticks by setting the first ytick to be empty
plt.yticks(ticks=[0] + tick_values[1:], labels=[""] + tick_labels[1:], fontsize=14, rotation=90)



# Setting X-axis title
ax.set_xlabel('UTM$_E$ [m]', fontsize=24)
ax.set_ylabel('UTM$_N$ [m]', fontsize=24)
ax.xaxis.labelpad = 10  # title standoff
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from tqdm import tqdm
import matplotlib

matplotlib.rcParams['font.family'] = 'Times New Roman'

def lin_to_dB(pow_lin):
    return 10*np.log10(np.array(pow_lin))


def aggregate_data(data, block_size):
    """
    Aggregates the data into larger blocks by averaging.

    Parameters:
    - data (np.array): 2D array of the data.
    - block_size (int): The size of the block to aggregate the data into.

    Returns:
    - np.array: Aggregated data.
    """
    # Define the new shape based on the block size
    shape = (data.shape[0] // block_size, data.shape[1] // block_size)
    aggregated_data = np.zeros(shape)

    for i in range(shape[0]):
        for j in range(shape[1]):
            aggregated_data[i, j] = np.mean(data[i*block_size:(i+1)*block_size, 
                                                 j*block_size:(j+1)*block_size])
    return aggregated_data

def draw_aggregated_squares(ax, power_data, duty_cycle_data, block_size):
    """
    Draws aggregated squares inside each pixel based on the aggregated duty cycle and power data.

    Parameters:
    - ax: The matplotlib axis to draw on.
    - power_data: 2D numpy array with the average power values.
    - duty_cycle_data: 2D numpy array with the duty cycle values.
    - block_size: The size of each block of aggregated data.
    """
    # Define the colormap for the average power
    power_cmap = plt.cm.jet

    # Normalize the power data for the colormap
    power_norm = mcolors.Normalize(vmin=np.min(power_data), vmax=np.max(power_data))

    # Calculate the side length of each aggregated block
    pixel_size = block_size

    # Iterate over the grid of aggregated data
    for i in tqdm(range(power_data.shape[0])):
        for j in range(power_data.shape[1]):
            # Calculate the size of the square based on the duty cycle
            square_side_length = (duty_cycle_data[i, j])**(1/1.1) * pixel_size

            # Calculate the color based on the average power
            square_color = power_cmap(power_norm(power_data[i, j]))

            # Calculate the coordinates of the bottom-left corner of the square
            square_x = j * pixel_size + (pixel_size - square_side_length) / 2
            square_y = i * pixel_size + (pixel_size - square_side_length) / 2

            # Draw the square for the duty cycle
            ax.add_patch(plt.Rectangle((square_x, square_y), square_side_length, square_side_length, 
                                       color=square_color))

            # Draw the white background for the block
            ax.add_patch(plt.Rectangle((j * pixel_size, i * pixel_size), pixel_size, pixel_size, 
                                       edgecolor='grey', facecolor='white', zorder=0))

# Example usage
fig, ax = plt.subplots(figsize=(10, 8))
ax.set_aspect('equal')

# Create some example data
power_data = lin_to_dB(np.load("signal_estimates_2160_2170_low_res_cov2.npy")).reshape(map_.shape)
print(power_data.shape)
duty_cycle_data = np.load("duty_cycle_2160_2170_idw.npy")/100
# The block size for aggregation
block_size = 14

# Aggregate the data
aggregated_power = aggregate_data(power_data, block_size)
aggregated_duty_cycle = aggregate_data(duty_cycle_data, block_size)

# Draw the aggregated squares
draw_aggregated_squares(ax, aggregated_power, aggregated_duty_cycle, block_size)

# Set the limits and turn off the axes
plt.xlim(0, aggregated_power.shape[1] * block_size)
plt.ylim(0, aggregated_power.shape[0] * block_size)
#plt.axis('off')

# Create a ScalarMappable with your colormap and normalization
sm = plt.cm.ScalarMappable(cmap=plt.cm.jet, norm=plt.Normalize(vmin=np.min(power_data), vmax=np.max(power_data)))

# Create the colorbar with a smaller size
cbar = plt.colorbar(sm, ax=ax, shrink=0.8, pad=0.02)  # adjust the shrink value as needed

# Set a larger font size for the colorbar label
cbar.set_label('Average Power [dBX]', size=20)  # adjust the size value as needed

# Increase font size of colorbar ticks
cbar.ax.tick_params(labelsize=19)
# Other plot settings...

# Show the plot
plt.tight_layout()
plt.title("2504-2544 MHz Spectrum Occupancy", fontsize=27)
N1 = map_.shape[0]
N2 = map_.shape[1]

en = SLC[column_map["axis"]]
UTM_long = np.linspace(en[0, 2], en[0, 3] - SLC[column_map["cellsize"]], N1)
UTM_lat = np.linspace(en[0, 0], en[0, 1] - SLC[column_map["cellsize"]], N2)
UTM_long = np.squeeze(UTM_long)
UTM_lat = np.squeeze(UTM_lat)
interval = 100
ax.set_xticks(range(0, len(UTM_lat), interval))
ax.set_xticklabels(UTM_lat[::interval], fontsize=14, rotation=0)

ax.set_yticks(range(0, len(UTM_long), interval))
ax.set_yticklabels(UTM_long[::interval], fontsize=14)



interval = max(1, len(UTM_lat) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_lat), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_lat[::interval]]

# Set the tick values and labels
plt.xticks(ticks=tick_values, labels=tick_labels, fontsize=14, rotation=0)

interval = max(1, len(UTM_long) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_long), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_long[::interval]]

# Adjust the overlapping xticks and yticks by setting the first ytick to be empty
plt.yticks(ticks=[0] + tick_values[1:], labels=[""] + tick_labels[1:], fontsize=14, rotation=90)



# Setting X-axis title
ax.set_xlabel('UTM$_E$ [m]', fontsize=24)
ax.set_ylabel('UTM$_N$ [m]', fontsize=24)
ax.xaxis.labelpad = 10  # title standoff
plt.show()


### Variation and Confidence

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from tqdm import tqdm
import matplotlib

matplotlib.rcParams['font.family'] = 'Times New Roman'

def lin_to_dB(pow_lin):
    return 10*np.log10(np.array(pow_lin))


def aggregate_data(data, block_size):
    """
    Aggregates the data into larger blocks by averaging.

    Parameters:
    - data (np.array): 2D array of the data.
    - block_size (int): The size of the block to aggregate the data into.

    Returns:
    - np.array: Aggregated data.
    """
    # Define the new shape based on the block size
    shape = (data.shape[0] // block_size, data.shape[1] // block_size)
    aggregated_data = np.zeros(shape)

    for i in range(shape[0]):
        for j in range(shape[1]):
            aggregated_data[i, j] = np.mean(data[i*block_size:(i+1)*block_size, 
                                                 j*block_size:(j+1)*block_size])
    return aggregated_data

def draw_aggregated_squares(ax, power_data, duty_cycle_data, block_size):
    """
    Draws aggregated squares inside each pixel based on the aggregated duty cycle and power data.

    Parameters:
    - ax: The matplotlib axis to draw on.
    - power_data: 2D numpy array with the average power values.
    - duty_cycle_data: 2D numpy array with the duty cycle values.
    - block_size: The size of each block of aggregated data.
    """
    # Define the colormap for the average power
    power_cmap = plt.cm.jet

    # Normalize the power data for the colormap
    power_norm = mcolors.Normalize(vmin=np.min(power_data), vmax=np.max(power_data))

    # Calculate the side length of each aggregated block
    pixel_size = block_size

    # Iterate over the grid of aggregated data
    for i in tqdm(range(power_data.shape[0])):
        for j in range(power_data.shape[1]):
            # Calculate the size of the square based on the duty cycle
            square_side_length = (duty_cycle_data[i, j])**(1/1.1) * pixel_size

            # Calculate the color based on the average power
            square_color = power_cmap(power_norm(power_data[i, j]))

            # Calculate the coordinates of the bottom-left corner of the square
            square_x = j * pixel_size + (pixel_size - square_side_length) / 2
            square_y = i * pixel_size + (pixel_size - square_side_length) / 2

            # Draw the square for the duty cycle
            ax.add_patch(plt.Rectangle((square_x, square_y), square_side_length, square_side_length, 
                                       color=square_color))

# Example usage
fig, ax = plt.subplots(figsize=(10, 8))
ax.set_aspect('equal')

# Create some example data
power_data = lin_to_dB(np.load("var_pred_2160_2170.npy")).reshape(map_.shape)
print(power_data.shape)
duty_cycle_data = np.load("confidence_level.npy")
# The block size for aggregation
block_size = 14

# Aggregate the data
aggregated_power = aggregate_data(power_data, block_size)
aggregated_duty_cycle = aggregate_data(duty_cycle_data, block_size)

# Draw the aggregated squares
draw_aggregated_squares(ax, aggregated_power, aggregated_duty_cycle, block_size)

# Set the limits and turn off the axes
plt.xlim(0, aggregated_power.shape[1] * block_size)
plt.ylim(0, aggregated_power.shape[0] * block_size)
#plt.axis('off')

# Create a ScalarMappable with your colormap and normalization
sm = plt.cm.ScalarMappable(cmap=plt.cm.jet, norm=plt.Normalize(vmin=np.min(power_data), vmax=np.max(power_data)))

# Create the colorbar with a smaller size
cbar = plt.colorbar(sm, ax=ax, shrink=0.8, pad=0.02)  # adjust the shrink value as needed

# Set a larger font size for the colorbar label
cbar.set_label('Signal Variation [dBX$^2$]', size=20)  # adjust the size value as needed

# Increase font size of colorbar ticks
cbar.ax.tick_params(labelsize=19)
# Other plot settings...

# Show the plot
plt.tight_layout()
plt.title("3470-3510 MHz Variation & Confidence", fontsize=27)
N1 = map_.shape[0]
N2 = map_.shape[1]

en = SLC[column_map["axis"]]
UTM_long = np.linspace(en[0, 2], en[0, 3] - SLC[column_map["cellsize"]], N1)
UTM_lat = np.linspace(en[0, 0], en[0, 1] - SLC[column_map["cellsize"]], N2)
UTM_long = np.squeeze(UTM_long)
UTM_lat = np.squeeze(UTM_lat)
interval = 100
ax.set_xticks(range(0, len(UTM_lat), interval))
ax.set_xticklabels(UTM_lat[::interval], fontsize=14, rotation=0)

ax.set_yticks(range(0, len(UTM_long), interval))
ax.set_yticklabels(UTM_long[::interval], fontsize=14)



interval = max(1, len(UTM_lat) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_lat), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_lat[::interval]]

# Set the tick values and labels
plt.xticks(ticks=tick_values, labels=tick_labels, fontsize=14, rotation=0)

interval = max(1, len(UTM_long) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_long), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_long[::interval]]

# Adjust the overlapping xticks and yticks by setting the first ytick to be empty
plt.yticks(ticks=[0] + tick_values[1:], labels=[""] + tick_labels[1:], fontsize=14, rotation=90)



# Setting X-axis title
ax.set_xlabel('UTM$_E$ [m]', fontsize=24)
ax.set_ylabel('UTM$_N$ [m]', fontsize=24)
ax.xaxis.labelpad = 10  # title standoff
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from tqdm import tqdm
import matplotlib

matplotlib.rcParams['font.family'] = 'Times New Roman'

def lin_to_dB(pow_lin):
    return 10*np.log10(np.array(pow_lin))


def aggregate_data(data, block_size):
    """
    Aggregates the data into larger blocks by averaging.

    Parameters:
    - data (np.array): 2D array of the data.
    - block_size (int): The size of the block to aggregate the data into.

    Returns:
    - np.array: Aggregated data.
    """
    # Define the new shape based on the block size
    shape = (data.shape[0] // block_size, data.shape[1] // block_size)
    aggregated_data = np.zeros(shape)

    for i in range(shape[0]):
        for j in range(shape[1]):
            aggregated_data[i, j] = np.mean(data[i*block_size:(i+1)*block_size, 
                                                 j*block_size:(j+1)*block_size])
    return aggregated_data

def draw_aggregated_squares(ax, power_data, duty_cycle_data, block_size):
    """
    Draws aggregated squares inside each pixel based on the aggregated duty cycle and power data.

    Parameters:
    - ax: The matplotlib axis to draw on.
    - power_data: 2D numpy array with the average power values.
    - duty_cycle_data: 2D numpy array with the duty cycle values.
    - block_size: The size of each block of aggregated data.
    """
    # Define the colormap for the average power
    power_cmap = plt.cm.jet

    # Normalize the power data for the colormap
    power_norm = mcolors.Normalize(vmin=np.min(power_data), vmax=np.max(power_data))

    # Calculate the side length of each aggregated block
    pixel_size = block_size

    # Iterate over the grid of aggregated data
    for i in tqdm(range(power_data.shape[0])):
        for j in range(power_data.shape[1]):
            # Calculate the size of the square based on the duty cycle
            square_side_length = (duty_cycle_data[i, j])**(1/1.1) * pixel_size

            # Calculate the color based on the average power
            square_color = power_cmap(power_norm(power_data[i, j]))

            # Calculate the coordinates of the bottom-left corner of the square
            square_x = j * pixel_size + (pixel_size - square_side_length) / 2
            square_y = i * pixel_size + (pixel_size - square_side_length) / 2

            # Draw the square for the duty cycle
            ax.add_patch(plt.Rectangle((square_x, square_y), square_side_length, square_side_length, 
                                       color=square_color))

# Example usage
fig, ax = plt.subplots(figsize=(10, 8))
ax.set_aspect('equal')

# Create some example data
power_data = lin_to_dB(np.load("var_pred_3470_3510_3.npy")).reshape(map_.shape)
print(power_data.shape)
duty_cycle_data = np.load("confidence_level.npy")
# The block size for aggregation
block_size = 14

# Aggregate the data
aggregated_power = aggregate_data(power_data, block_size)
aggregated_duty_cycle = aggregate_data(duty_cycle_data, block_size)

# Draw the aggregated squares
draw_aggregated_squares(ax, aggregated_power, aggregated_duty_cycle, block_size)

# Set the limits and turn off the axes
plt.xlim(0, aggregated_power.shape[1] * block_size)
plt.ylim(0, aggregated_power.shape[0] * block_size)
#plt.axis('off')

# Create a ScalarMappable with your colormap and normalization
sm = plt.cm.ScalarMappable(cmap=plt.cm.jet, norm=plt.Normalize(vmin=np.min(power_data), vmax=np.max(power_data)))

# Create the colorbar with a smaller size
cbar = plt.colorbar(sm, ax=ax, shrink=0.8, pad=0.02)  # adjust the shrink value as needed

# Set a larger font size for the colorbar label
cbar.set_label('Signal Variation [dBX$^2$]', size=20)  # adjust the size value as needed

# Increase font size of colorbar ticks
cbar.ax.tick_params(labelsize=19)
# Other plot settings...

# Show the plot
plt.tight_layout()
plt.title("3470-3510 MHz Variation & Confidence", fontsize=27)
N1 = map_.shape[0]
N2 = map_.shape[1]

en = SLC[column_map["axis"]]
UTM_long = np.linspace(en[0, 2], en[0, 3] - SLC[column_map["cellsize"]], N1)
UTM_lat = np.linspace(en[0, 0], en[0, 1] - SLC[column_map["cellsize"]], N2)
UTM_long = np.squeeze(UTM_long)
UTM_lat = np.squeeze(UTM_lat)
interval = 100
ax.set_xticks(range(0, len(UTM_lat), interval))
ax.set_xticklabels(UTM_lat[::interval], fontsize=14, rotation=0)

ax.set_yticks(range(0, len(UTM_long), interval))
ax.set_yticklabels(UTM_long[::interval], fontsize=14)



interval = max(1, len(UTM_lat) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_lat), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_lat[::interval]]

# Set the tick values and labels
plt.xticks(ticks=tick_values, labels=tick_labels, fontsize=14, rotation=0)

interval = max(1, len(UTM_long) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_long), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_long[::interval]]

# Adjust the overlapping xticks and yticks by setting the first ytick to be empty
plt.yticks(ticks=[0] + tick_values[1:], labels=[""] + tick_labels[1:], fontsize=14, rotation=90)



# Setting X-axis title
ax.set_xlabel('UTM$_E$ [m]', fontsize=24)
ax.set_ylabel('UTM$_N$ [m]', fontsize=24)
ax.xaxis.labelpad = 10  # title standoff
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from tqdm import tqdm
import matplotlib

matplotlib.rcParams['font.family'] = 'Times New Roman'

def lin_to_dB(pow_lin):
    return 10*np.log10(np.array(pow_lin))


def aggregate_data(data, block_size):
    """
    Aggregates the data into larger blocks by averaging.

    Parameters:
    - data (np.array): 2D array of the data.
    - block_size (int): The size of the block to aggregate the data into.

    Returns:
    - np.array: Aggregated data.
    """
    # Define the new shape based on the block size
    shape = (data.shape[0] // block_size, data.shape[1] // block_size)
    aggregated_data = np.zeros(shape)

    for i in range(shape[0]):
        for j in range(shape[1]):
            aggregated_data[i, j] = np.mean(data[i*block_size:(i+1)*block_size, 
                                                 j*block_size:(j+1)*block_size])
    return aggregated_data

def draw_aggregated_squares(ax, power_data, duty_cycle_data, block_size):
    """
    Draws aggregated squares inside each pixel based on the aggregated duty cycle and power data.

    Parameters:
    - ax: The matplotlib axis to draw on.
    - power_data: 2D numpy array with the average power values.
    - duty_cycle_data: 2D numpy array with the duty cycle values.
    - block_size: The size of each block of aggregated data.
    """
    # Define the colormap for the average power
    power_cmap = plt.cm.jet

    # Normalize the power data for the colormap
    power_norm = mcolors.Normalize(vmin=np.min(power_data), vmax=np.max(power_data))

    # Calculate the side length of each aggregated block
    pixel_size = block_size

    # Iterate over the grid of aggregated data
    for i in tqdm(range(power_data.shape[0])):
        for j in range(power_data.shape[1]):
            # Calculate the size of the square based on the duty cycle
            square_side_length = (duty_cycle_data[i, j])**(1/1.1) * pixel_size

            # Calculate the color based on the average power
            square_color = power_cmap(power_norm(power_data[i, j]))

            # Calculate the coordinates of the bottom-left corner of the square
            square_x = j * pixel_size + (pixel_size - square_side_length) / 2
            square_y = i * pixel_size + (pixel_size - square_side_length) / 2

            # Draw the square for the duty cycle
            ax.add_patch(plt.Rectangle((square_x, square_y), square_side_length, square_side_length, 
                                       color=square_color))

# Example usage
fig, ax = plt.subplots(figsize=(10, 8))
ax.set_aspect('equal')

# Create some example data
power_data = lin_to_dB(np.load("var_pred_3610_3650_2.npy")).reshape(map_.shape)
print(power_data.shape)
duty_cycle_data = np.load("confidence_level.npy")
# The block size for aggregation
block_size = 14

# Aggregate the data
aggregated_power = aggregate_data(power_data, block_size)
aggregated_duty_cycle = aggregate_data(duty_cycle_data, block_size)

# Draw the aggregated squares
draw_aggregated_squares(ax, aggregated_power, aggregated_duty_cycle, block_size)

# Set the limits and turn off the axes
plt.xlim(0, aggregated_power.shape[1] * block_size)
plt.ylim(0, aggregated_power.shape[0] * block_size)
#plt.axis('off')

# Create a ScalarMappable with your colormap and normalization
sm = plt.cm.ScalarMappable(cmap=plt.cm.jet, norm=plt.Normalize(vmin=np.min(power_data), vmax=np.max(power_data)))

# Create the colorbar with a smaller size
cbar = plt.colorbar(sm, ax=ax, shrink=0.8, pad=0.02)  # adjust the shrink value as needed

# Set a larger font size for the colorbar label
cbar.set_label('Signal Variation [dBX$^2$]', size=20)  # adjust the size value as needed

# Increase font size of colorbar ticks
cbar.ax.tick_params(labelsize=19)
# Other plot settings...

# Show the plot
plt.tight_layout()
plt.title("3610-3650 MHz Variation & Confidence", fontsize=27)
N1 = map_.shape[0]
N2 = map_.shape[1]

en = SLC[column_map["axis"]]
UTM_long = np.linspace(en[0, 2], en[0, 3] - SLC[column_map["cellsize"]], N1)
UTM_lat = np.linspace(en[0, 0], en[0, 1] - SLC[column_map["cellsize"]], N2)
UTM_long = np.squeeze(UTM_long)
UTM_lat = np.squeeze(UTM_lat)
interval = 100
ax.set_xticks(range(0, len(UTM_lat), interval))
ax.set_xticklabels(UTM_lat[::interval], fontsize=14, rotation=0)

ax.set_yticks(range(0, len(UTM_long), interval))
ax.set_yticklabels(UTM_long[::interval], fontsize=14)



interval = max(1, len(UTM_lat) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_lat), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_lat[::interval]]

# Set the tick values and labels
plt.xticks(ticks=tick_values, labels=tick_labels, fontsize=14, rotation=0)

interval = max(1, len(UTM_long) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_long), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_long[::interval]]

# Adjust the overlapping xticks and yticks by setting the first ytick to be empty
plt.yticks(ticks=[0] + tick_values[1:], labels=[""] + tick_labels[1:], fontsize=14, rotation=90)



# Setting X-axis title
ax.set_xlabel('UTM$_E$ [m]', fontsize=24)
ax.set_ylabel('UTM$_N$ [m]', fontsize=24)
ax.xaxis.labelpad = 10  # title standoff
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from tqdm import tqdm
import matplotlib

matplotlib.rcParams['font.family'] = 'Times New Roman'

def lin_to_dB(pow_lin):
    return 10*np.log10(np.array(pow_lin))


def aggregate_data(data, block_size):
    """
    Aggregates the data into larger blocks by averaging.

    Parameters:
    - data (np.array): 2D array of the data.
    - block_size (int): The size of the block to aggregate the data into.

    Returns:
    - np.array: Aggregated data.
    """
    # Define the new shape based on the block size
    shape = (data.shape[0] // block_size, data.shape[1] // block_size)
    aggregated_data = np.zeros(shape)

    for i in range(shape[0]):
        for j in range(shape[1]):
            aggregated_data[i, j] = np.mean(data[i*block_size:(i+1)*block_size, 
                                                 j*block_size:(j+1)*block_size])
    return aggregated_data

def draw_aggregated_squares(ax, power_data, duty_cycle_data, block_size):
    """
    Draws aggregated squares inside each pixel based on the aggregated duty cycle and power data.

    Parameters:
    - ax: The matplotlib axis to draw on.
    - power_data: 2D numpy array with the average power values.
    - duty_cycle_data: 2D numpy array with the duty cycle values.
    - block_size: The size of each block of aggregated data.
    """
    # Define the colormap for the average power
    power_cmap = plt.cm.jet

    # Normalize the power data for the colormap
    power_norm = mcolors.Normalize(vmin=np.min(power_data), vmax=np.max(power_data))

    # Calculate the side length of each aggregated block
    pixel_size = block_size

    # Iterate over the grid of aggregated data
    for i in tqdm(range(power_data.shape[0])):
        for j in range(power_data.shape[1]):
            # Calculate the size of the square based on the duty cycle
            square_side_length = (duty_cycle_data[i, j])**(1/1.1) * pixel_size

            # Calculate the color based on the average power
            square_color = power_cmap(power_norm(power_data[i, j]))

            # Calculate the coordinates of the bottom-left corner of the square
            square_x = j * pixel_size + (pixel_size - square_side_length) / 2
            square_y = i * pixel_size + (pixel_size - square_side_length) / 2

            # Draw the square for the duty cycle
            ax.add_patch(plt.Rectangle((square_x, square_y), square_side_length, square_side_length, 
                                       color=square_color))

# Example usage
fig, ax = plt.subplots(figsize=(10, 8))
ax.set_aspect('equal')

# Create some example data
power_data = lin_to_dB(np.load("var_pred_2160_2170_3.npy")).reshape(map_.shape)
power_data[np.isnan(power_data)]=0
print(power_data.shape)
duty_cycle_data = np.load("confidence_level.npy")
# The block size for aggregation
block_size = 14

# Aggregate the data
aggregated_power = aggregate_data(power_data, block_size)
aggregated_duty_cycle = aggregate_data(duty_cycle_data, block_size)

# Draw the aggregated squares
draw_aggregated_squares(ax, aggregated_power, aggregated_duty_cycle, block_size)

# Set the limits and turn off the axes
plt.xlim(0, aggregated_power.shape[1] * block_size)
plt.ylim(0, aggregated_power.shape[0] * block_size)
#plt.axis('off')

# Create a ScalarMappable with your colormap and normalization
sm = plt.cm.ScalarMappable(cmap=plt.cm.jet, norm=plt.Normalize(vmin=np.min(power_data), vmax=np.max(power_data)))

# Create the colorbar with a smaller size
cbar = plt.colorbar(sm, ax=ax, shrink=0.8, pad=0.02)  # adjust the shrink value as needed

# Set a larger font size for the colorbar label
cbar.set_label('Signal Variation [dBX$^2$]', size=20)  # adjust the size value as needed

# Increase font size of colorbar ticks
cbar.ax.tick_params(labelsize=19)
# Other plot settings...

# Show the plot
plt.tight_layout()
plt.title("2160.5-2169.5 MHz Variation & Confidence", fontsize=27)
N1 = map_.shape[0]
N2 = map_.shape[1]

en = SLC[column_map["axis"]]
UTM_long = np.linspace(en[0, 2], en[0, 3] - SLC[column_map["cellsize"]], N1)
UTM_lat = np.linspace(en[0, 0], en[0, 1] - SLC[column_map["cellsize"]], N2)
UTM_long = np.squeeze(UTM_long)
UTM_lat = np.squeeze(UTM_lat)
interval = 100
ax.set_xticks(range(0, len(UTM_lat), interval))
ax.set_xticklabels(UTM_lat[::interval], fontsize=14, rotation=0)

ax.set_yticks(range(0, len(UTM_long), interval))
ax.set_yticklabels(UTM_long[::interval], fontsize=14)



interval = max(1, len(UTM_lat) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_lat), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_lat[::interval]]

# Set the tick values and labels
plt.xticks(ticks=tick_values, labels=tick_labels, fontsize=14, rotation=0)

interval = max(1, len(UTM_long) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_long), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_long[::interval]]

# Adjust the overlapping xticks and yticks by setting the first ytick to be empty
plt.yticks(ticks=[0] + tick_values[1:], labels=[""] + tick_labels[1:], fontsize=14, rotation=90)



# Setting X-axis title
ax.set_xlabel('UTM$_E$ [m]', fontsize=24)
ax.set_ylabel('UTM$_N$ [m]', fontsize=24)
ax.xaxis.labelpad = 10  # title standoff
plt.show()


In [ ]:
power_data

### Combine Terrain with Occupancy Power

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
from tqdm import tqdm
import matplotlib

# Set font style
matplotlib.rcParams['font.family'] = 'Times New Roman'

# Function to convert linear power to dB
def lin_to_dB(pow_lin):
    return 10 * np.log10(np.array(pow_lin))

# Function to aggregate data into larger blocks by averaging
def aggregate_data(data, block_size):
    shape = (data.shape[0] // block_size, data.shape[1] // block_size)
    aggregated_data = np.zeros(shape)

    for i in range(shape[0]):
        for j in range(shape[1]):
            aggregated_data[i, j] = np.mean(data[i*block_size:(i+1)*block_size, 
                                                 j*block_size:(j+1)*block_size])
    return aggregated_data

# Function to draw aggregated squares based on power and duty cycle data
def draw_aggregated_squares(ax, power_data, duty_cycle_data, block_size):
    power_cmap = plt.cm.jet
    power_norm = mcolors.Normalize(vmin=np.min(power_data), vmax=np.max(power_data))
    pixel_size = block_size

    for i in tqdm(range(power_data.shape[0])):
        for j in range(power_data.shape[1]):
            square_side_length = (duty_cycle_data[i, j])**(1/1.1) * pixel_size
            square_color = power_cmap(power_norm(power_data[i, j]))
            square_x = j * pixel_size + (pixel_size - square_side_length) / 2
            square_y = i * pixel_size + (pixel_size - square_side_length) / 2
            ax.add_patch(plt.Rectangle((square_x, square_y), square_side_length, square_side_length, 
                                       color=square_color))

# Placeholder for actual data loading and preparation
# ... (Load your X, Y, bldg, dem, power_data, duty_cycle_data here) ...
import os
import scipy.io as sio
import matplotlib.pyplot as plt
map_folderdir = "./"
directory = os.listdir(map_folderdir)
flag = 0
for fname in directory:
    if "SLCmap" in fname:
        map_file = os.path.join(map_folderdir, fname)
        flag = 1

if flag == 0:
    errorMessage = 'Error: The file does not exist in the folder:\n ' + map_folderdir
    warnings.warn(errorMessage)

print('Now reading ' + map_file + "\n")
x = sio.loadmat(map_file)
map_struct = x['SLC']

# Define a new struct named SLC
SLC = map_struct[0][0]
column_map = dict(zip([name for name in SLC.dtype.names], [i for i in range(len(SLC.dtype.names))]))

map_ = SLC[column_map["dem"]] + 0.3048 * SLC[column_map["hybrid_bldg"]]
x = np.linspace(0, map_.shape[1]//10+1, map_.shape[1]//10 + 1 , endpoint=False)
y = np.linspace(0, map_.shape[0]//10+1, map_.shape[0]//10 + 1, endpoint=False)
X, Y = np.meshgrid(x, y)
dem = SLC[column_map["dem"]]
bldg = 0.3048 * SLC[column_map["hybrid_bldg"]]
#map_ = SLC[column_map["dem"]] + 0.3048 * SLC[column_map["hybrid_bldg"]]

def replace_border_zeros(arr):
    rows, cols = arr.shape

    # Function to find nearest non-zero for a given position
    def nearest_non_zero(i, j):
        for d in range(1, max(rows, cols)):
            for di in range(-d, d + 1):
                for dj in range(-d, d + 1):
                    if 0 <= i + di < rows and 0 <= j + dj < cols and arr[i + di, j + dj] != 0:
                        return arr[i + di, j + dj]
        return 0  # In case there are no non-zero neighbors

    # Replace zeros on top and bottom rows
    for j in range(cols):
        if arr[0, j] == 0:
            arr[0, j] = nearest_non_zero(0, j)
        if arr[rows - 1, j] == 0:
            arr[rows - 1, j] = nearest_non_zero(rows - 1, j)

    # Replace zeros on left and right columns
    for i in range(rows):
        if arr[i, 0] == 0:
            arr[i, 0] = nearest_non_zero(i, 0)
        if arr[i, cols - 1] == 0:
            arr[i, cols - 1] = nearest_non_zero(i, cols - 1)

    return arr

# Example array

# Replace border zeros
dem = replace_border_zeros(dem)
map_ = map_[::10, ::10]
dem = dem[::10, ::10]
bldg = bldg[::10, ::10]
power_data = lin_to_dB(np.load("signal_estimates_3610_3650_low_res_cov3.npy")).reshape(map_.shape)
print(power_data.shape)
duty_cycle_data = np.load("duty_cycle_3610_3650_idw2.npy")/100
block_size=14
# Initialize the plot
fig, ax = plt.subplots(figsize=(13, 10))
ax.set_aspect('equal')

# Aggregate the data
aggregated_power = aggregate_data(power_data, block_size)
aggregated_duty_cycle = aggregate_data(duty_cycle_data, block_size)
aggregated_dem = aggregate_data(dem, block_size)
aggregated_bldg = aggregate_data(bldg, block_size)
aggregated_X = aggregate_data(X, block_size)
aggregated_Y = aggregate_data(Y, block_size)

# Draw the aggregated squares (spectrum occupancy)
draw_aggregated_squares(ax, aggregated_power, aggregated_duty_cycle, block_size)

print(X.shape)
print(Y.shape)
print(dem.shape)

# Overlay contour plot (terrain elevation)
contours = plt.contour(X, Y, dem, 10, cmap='gist_earth', linewidths=3.5)
plt.clabel(contours, inline=True, fontsize=14)

# Create colorbar for terrain elevation
cbar_dem = plt.colorbar(contours, label='Terrain Elevation [m]', location='left')
cbar_dem.set_label('Terrain Elevation [m]', size=20)
cbar_dem.ax.tick_params(labelsize=19)

# Create colorbar for average power
sm = plt.cm.ScalarMappable(cmap=plt.cm.jet, norm=plt.Normalize(vmin=np.min(power_data), vmax=np.max(power_data)))
cbar_power = plt.colorbar(sm, ax=ax, shrink=0.8, pad=0.02)
cbar_power.set_label('Average Power [dBX]', size=20)
cbar_power.ax.tick_params(labelsize=19)

# Set axis labels and titles
ax.set_xlabel('UTM$_E$ [m]', fontsize=24)
ax.set_ylabel('UTM$_N$ [m]', fontsize=24)
plt.title("3610-3650 MHz Spectrum Occupancy and Terrain Elevation", fontsize=27)

N1 = map_.shape[0]
N2 = map_.shape[1]

en = SLC[column_map["axis"]]
UTM_long = np.linspace(en[0, 2], en[0, 3] - SLC[column_map["cellsize"]], N1)
UTM_lat = np.linspace(en[0, 0], en[0, 1] - SLC[column_map["cellsize"]], N2)
UTM_long = np.squeeze(UTM_long)
UTM_lat = np.squeeze(UTM_lat)
interval = 100
ax.set_xticks(range(0, len(UTM_lat), interval))
ax.set_xticklabels(UTM_lat[::interval], fontsize=14, rotation=0)

ax.set_yticks(range(0, len(UTM_long), interval))
ax.set_yticklabels(UTM_long[::interval], fontsize=14)



interval = max(1, len(UTM_lat) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_lat), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_lat[::interval]]

# Set the tick values and labels
plt.xticks(ticks=tick_values, labels=tick_labels, fontsize=14, rotation=0)

interval = max(1, len(UTM_long) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_long), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_long[::interval]]

# Adjust the overlapping xticks and yticks by setting the first ytick to be empty
plt.yticks(ticks=[0] + tick_values[1:], labels=[""] + tick_labels[1:], fontsize=14, rotation=90)


# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
from tqdm import tqdm
import matplotlib

# Set font style
matplotlib.rcParams['font.family'] = 'Times New Roman'

# Function to convert linear power to dB
def lin_to_dB(pow_lin):
    return 10 * np.log10(np.array(pow_lin))

# Function to aggregate data into larger blocks by averaging
def aggregate_data(data, block_size):
    shape = (data.shape[0] // block_size, data.shape[1] // block_size)
    aggregated_data = np.zeros(shape)

    for i in range(shape[0]):
        for j in range(shape[1]):
            aggregated_data[i, j] = np.mean(data[i*block_size:(i+1)*block_size, 
                                                 j*block_size:(j+1)*block_size])
    return aggregated_data

# Function to draw aggregated squares based on power and duty cycle data
def draw_aggregated_squares(ax, power_data, duty_cycle_data, block_size):
    power_cmap = plt.cm.jet
    power_norm = mcolors.Normalize(vmin=np.min(power_data), vmax=np.max(power_data))
    pixel_size = block_size

    for i in tqdm(range(power_data.shape[0])):
        for j in range(power_data.shape[1]):
            square_side_length = (duty_cycle_data[i, j])**(1/1.1) * pixel_size
            square_color = power_cmap(power_norm(power_data[i, j]))
            square_x = j * pixel_size + (pixel_size - square_side_length) / 2
            square_y = i * pixel_size + (pixel_size - square_side_length) / 2
            # Inside the draw_aggregated_squares function
            ax.add_patch(plt.Rectangle((square_x, square_y), square_side_length, square_side_length, 
                                       color=square_color, alpha=0.5)) # Add transparency


# Placeholder for actual data loading and preparation
# ... (Load your X, Y, bldg, dem, power_data, duty_cycle_data here) ...
import os
import scipy.io as sio
import matplotlib.pyplot as plt
map_folderdir = "./"
directory = os.listdir(map_folderdir)
flag = 0
for fname in directory:
    if "SLCmap" in fname:
        map_file = os.path.join(map_folderdir, fname)
        flag = 1

if flag == 0:
    errorMessage = 'Error: The file does not exist in the folder:\n ' + map_folderdir
    warnings.warn(errorMessage)

print('Now reading ' + map_file + "\n")
x = sio.loadmat(map_file)
map_struct = x['SLC']

# Define a new struct named SLC
SLC = map_struct[0][0]
column_map = dict(zip([name for name in SLC.dtype.names], [i for i in range(len(SLC.dtype.names))]))

map_ = SLC[column_map["dem"]] + 0.3048 * SLC[column_map["hybrid_bldg"]]
map_ = map_[:5200, :5800]
x = np.linspace(0, map_.shape[1]//10, map_.shape[1]//10 , endpoint=False)
y = np.linspace(0, map_.shape[0]//10, map_.shape[0]//10, endpoint=False)
X, Y = np.meshgrid(x, y)
# After defining the X and Y grids
print(f"X grid shape: {X.shape}")
print(f"Y grid shape: {Y.shape}")
print(f"Power data shape: {aggregated_power.shape}")
print(f"Building data shape: {aggregated_bldg.shape}")

# Check the first and last few coordinates to see if they align
print(f"X grid start and end: {X[0, 0]}, {X[-1, -1]}")
print(f"Y grid start and end: {Y[0, 0]}, {Y[-1, -1]}")
print(f"Power data start and end values: {aggregated_power[0, 0]}, {aggregated_power[-1, -1]}")
print(f"Building data start and end values: {aggregated_bldg[0, 0]}, {aggregated_bldg[-1, -1]}")

dem = SLC[column_map["dem"]]
dem = dem[:5200, :5800]
bldg = 0.3048 * SLC[column_map["hybrid_bldg"]]
bldg = bldg[:5200, :5800]

#map_ = SLC[column_map["dem"]] + 0.3048 * SLC[column_map["hybrid_bldg"]]

def replace_border_zeros(arr):
    rows, cols = arr.shape

    # Function to find nearest non-zero for a given position
    def nearest_non_zero(i, j):
        for d in range(1, max(rows, cols)):
            for di in range(-d, d + 1):
                for dj in range(-d, d + 1):
                    if 0 <= i + di < rows and 0 <= j + dj < cols and arr[i + di, j + dj] != 0:
                        return arr[i + di, j + dj]
        return 0  # In case there are no non-zero neighbors

    # Replace zeros on top and bottom rows
    for j in range(cols):
        if arr[0, j] == 0:
            arr[0, j] = nearest_non_zero(0, j)
        if arr[rows - 1, j] == 0:
            arr[rows - 1, j] = nearest_non_zero(rows - 1, j)

    # Replace zeros on left and right columns
    for i in range(rows):
        if arr[i, 0] == 0:
            arr[i, 0] = nearest_non_zero(i, 0)
        if arr[i, cols - 1] == 0:
            arr[i, cols - 1] = nearest_non_zero(i, cols - 1)

    return arr

# Example array

# Replace border zeros
dem = replace_border_zeros(dem)
map_ = map_[::10, ::10]
dem = dem[::10, ::10]
bldg = bldg[::10, ::10]
power_data = lin_to_dB(np.load("signal_estimates_2160_2170_low_res_cov2.npy")).reshape((521, 585))[:520, :580]
print(power_data.shape)
duty_cycle_data = np.load("duty_cycle_2160_2170_idw2.npy")/100
duty_cycle_data = duty_cycle_data[:520, :580]
block_size=10
# Initialize the plot
fig, ax = plt.subplots(figsize=(13, 10))

# Aggregate the data
aggregated_power = aggregate_data(power_data, block_size)
aggregated_duty_cycle = aggregate_data(duty_cycle_data, block_size)
aggregated_dem = aggregate_data(dem, block_size)
aggregated_bldg = aggregate_data(bldg, block_size)
aggregated_X = aggregate_data(X, block_size)
aggregated_Y = aggregate_data(Y, block_size)

plt.contourf(X, Y, bldg, 100, cmap='gist_gray', alpha=0.5) # Adjust alpha for transparency

# Draw the aggregated squares (spectrum occupancy)
draw_aggregated_squares(ax, aggregated_power, aggregated_duty_cycle, block_size)

# Create colorbar for average power
sm = plt.cm.ScalarMappable(cmap=plt.cm.jet, norm=plt.Normalize(vmin=np.min(power_data), vmax=np.max(power_data)))
cbar_power = plt.colorbar(sm, ax=ax, shrink=1, pad=0.02)
cbar_power.ax.tick_params(labelsize=19)
cbar_power.set_label('Average Power [dBX]', size=20)  # adjust the size value as needed

# Set axis labels and titles
ax.set_xlabel('UTM$_E$ [m]', fontsize=24)
ax.set_ylabel('UTM$_N$ [m]', fontsize=24)
plt.title("2160.5-2169.5 MHz Spectrum Occupancy", fontsize=27)

N1 = map_.shape[0]
N2 = map_.shape[1]

en = SLC[column_map["axis"]]
UTM_long = np.linspace(en[0, 2], en[0, 3] - SLC[column_map["cellsize"]], N1)
UTM_lat = np.linspace(en[0, 0], en[0, 1] - SLC[column_map["cellsize"]], N2)
UTM_long = np.squeeze(UTM_long)
UTM_lat = np.squeeze(UTM_lat)
interval = 100
ax.set_xticks(range(0, len(UTM_lat), interval))
ax.set_xticklabels(UTM_lat[::interval], fontsize=14, rotation=0)

ax.set_yticks(range(0, len(UTM_long), interval))
ax.set_yticklabels(UTM_long[::interval], fontsize=14)



interval = max(1, len(UTM_lat) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_lat), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_lat[::interval]]

# Set the tick values and labels
plt.xticks(ticks=tick_values, labels=tick_labels, fontsize=14, rotation=0)

interval = max(1, len(UTM_long) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_long), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_long[::interval]]

# Adjust the overlapping xticks and yticks by setting the first ytick to be empty
plt.yticks(ticks=[0] + tick_values[1:], labels=[""] + tick_labels[1:], fontsize=14, rotation=90)
# First scatter plot for the edges (larger size, different color)
scatter = plt.scatter(data_points[:, 0], data_points[:, 1], c='purple', s=40, edgecolors='white', linewidths=2, label='Data Collection Points Edges')


ax.set_xlim([0, map_.shape[1]])
ax.set_ylim([0, map_.shape[0]])
ax.set_aspect('equal')

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
from tqdm import tqdm
import matplotlib

# Set font style
matplotlib.rcParams['font.family'] = 'Times New Roman'

# Function to convert linear power to dB
def lin_to_dB(pow_lin):
    return 10 * np.log10(np.array(pow_lin))

# Function to aggregate data into larger blocks by averaging
def aggregate_data(data, block_size):
    shape = (data.shape[0] // block_size, data.shape[1] // block_size)
    aggregated_data = np.zeros(shape)

    for i in range(shape[0]):
        for j in range(shape[1]):
            aggregated_data[i, j] = np.mean(data[i*block_size:(i+1)*block_size, 
                                                 j*block_size:(j+1)*block_size])
    return aggregated_data

# Function to draw aggregated squares based on power and duty cycle data
def draw_aggregated_squares(ax, power_data, duty_cycle_data, block_size):
    power_cmap = plt.cm.jet
    power_norm = mcolors.Normalize(vmin=np.min(power_data), vmax=np.max(power_data))
    pixel_size = block_size

    for i in tqdm(range(power_data.shape[0])):
        for j in range(power_data.shape[1]):
            square_side_length = (duty_cycle_data[i, j])**(1/1.1) * pixel_size
            square_color = power_cmap(power_norm(power_data[i, j]))
            square_x = j * pixel_size + (pixel_size - square_side_length) / 2
            square_y = i * pixel_size + (pixel_size - square_side_length) / 2
            # Inside the draw_aggregated_squares function
            ax.add_patch(plt.Rectangle((square_x, square_y), square_side_length, square_side_length, 
                                       color=square_color, alpha=0.5)) # Add transparency


import os
import scipy.io as sio
import matplotlib.pyplot as plt
map_folderdir = "./"
directory = os.listdir(map_folderdir)
flag = 0
for fname in directory:
    if "SLCmap" in fname:
        map_file = os.path.join(map_folderdir, fname)
        flag = 1

if flag == 0:
    errorMessage = 'Error: The file does not exist in the folder:\n ' + map_folderdir
    warnings.warn(errorMessage)

print('Now reading ' + map_file + "\n")
x = sio.loadmat(map_file)
map_struct = x['SLC']

# Define a new struct named SLC
SLC = map_struct[0][0]
column_map = dict(zip([name for name in SLC.dtype.names], [i for i in range(len(SLC.dtype.names))]))

map_ = SLC[column_map["dem"]] + 0.3048 * SLC[column_map["hybrid_bldg"]]
map_ = map_[:5200, :5800]
x = np.linspace(0, map_.shape[1]//10, map_.shape[1]//10 , endpoint=False)
y = np.linspace(0, map_.shape[0]//10, map_.shape[0]//10, endpoint=False)
X, Y = np.meshgrid(x, y)
# After defining the X and Y grids
print(f"X grid shape: {X.shape}")
print(f"Y grid shape: {Y.shape}")
print(f"Power data shape: {aggregated_power.shape}")
print(f"Building data shape: {aggregated_bldg.shape}")

# Check the first and last few coordinates to see if they align
print(f"X grid start and end: {X[0, 0]}, {X[-1, -1]}")
print(f"Y grid start and end: {Y[0, 0]}, {Y[-1, -1]}")
print(f"Power data start and end values: {aggregated_power[0, 0]}, {aggregated_power[-1, -1]}")
print(f"Building data start and end values: {aggregated_bldg[0, 0]}, {aggregated_bldg[-1, -1]}")

dem = SLC[column_map["dem"]]
dem = dem[:5200, :5800]
bldg = 0.3048 * SLC[column_map["hybrid_bldg"]]
bldg = bldg[:5200, :5800]

#map_ = SLC[column_map["dem"]] + 0.3048 * SLC[column_map["hybrid_bldg"]]

def replace_border_zeros(arr):
    rows, cols = arr.shape

    # Function to find nearest non-zero for a given position
    def nearest_non_zero(i, j):
        for d in range(1, max(rows, cols)):
            for di in range(-d, d + 1):
                for dj in range(-d, d + 1):
                    if 0 <= i + di < rows and 0 <= j + dj < cols and arr[i + di, j + dj] != 0:
                        return arr[i + di, j + dj]
        return 0  # In case there are no non-zero neighbors

    # Replace zeros on top and bottom rows
    for j in range(cols):
        if arr[0, j] == 0:
            arr[0, j] = nearest_non_zero(0, j)
        if arr[rows - 1, j] == 0:
            arr[rows - 1, j] = nearest_non_zero(rows - 1, j)

    # Replace zeros on left and right columns
    for i in range(rows):
        if arr[i, 0] == 0:
            arr[i, 0] = nearest_non_zero(i, 0)
        if arr[i, cols - 1] == 0:
            arr[i, cols - 1] = nearest_non_zero(i, cols - 1)

    return arr

# Example array

# Replace border zeros
dem = replace_border_zeros(dem)
map_ = map_[::10, ::10]
dem = dem[::10, ::10]
bldg = bldg[::10, ::10]
power_data = lin_to_dB(np.load("var_pred_2160_2170_3.npy")).reshape((521, 585))[:520, :580]
power_data[np.isnan(power_data)]=0
print(power_data.shape)
duty_cycle_data = np.load("confidence_level.npy")
duty_cycle_data = duty_cycle_data[:520, :580]

block_size=10
# Initialize the plot
fig, ax = plt.subplots(figsize=(13, 10))

# Aggregate the data
aggregated_power = aggregate_data(power_data, block_size)
aggregated_duty_cycle = aggregate_data(duty_cycle_data, block_size)
aggregated_dem = aggregate_data(dem, block_size)
aggregated_bldg = aggregate_data(bldg, block_size)
aggregated_X = aggregate_data(X, block_size)
aggregated_Y = aggregate_data(Y, block_size)

plt.contourf(X, Y, bldg, 100, cmap='gist_gray', alpha=0.5) # Adjust alpha for transparency

# Draw the aggregated squares (spectrum occupancy)
draw_aggregated_squares(ax, aggregated_power, aggregated_duty_cycle, block_size)

# Create colorbar for average power
sm = plt.cm.ScalarMappable(cmap=plt.cm.jet, norm=plt.Normalize(vmin=np.min(power_data), vmax=np.max(power_data)))
cbar_power = plt.colorbar(sm, ax=ax, shrink=1, pad=0.02)
cbar_power.ax.tick_params(labelsize=19)
cbar_power.set_label('Signal Variance [dBX^2]', size=20)  # adjust the size value as needed

# Set axis labels and titles
ax.set_xlabel('UTM$_E$ [m]', fontsize=24)
ax.set_ylabel('UTM$_N$ [m]', fontsize=24)
plt.title("2160.5-2169.5 MHz Variation & Confidence", fontsize=27)

N1 = map_.shape[0]
N2 = map_.shape[1]

en = SLC[column_map["axis"]]
UTM_long = np.linspace(en[0, 2], en[0, 3] - SLC[column_map["cellsize"]], N1)
UTM_lat = np.linspace(en[0, 0], en[0, 1] - SLC[column_map["cellsize"]], N2)
UTM_long = np.squeeze(UTM_long)
UTM_lat = np.squeeze(UTM_lat)
interval = 100
ax.set_xticks(range(0, len(UTM_lat), interval))
ax.set_xticklabels(UTM_lat[::interval], fontsize=14, rotation=0)

ax.set_yticks(range(0, len(UTM_long), interval))
ax.set_yticklabels(UTM_long[::interval], fontsize=14)



interval = max(1, len(UTM_lat) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_lat), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_lat[::interval]]

# Set the tick values and labels
plt.xticks(ticks=tick_values, labels=tick_labels, fontsize=14, rotation=0)

interval = max(1, len(UTM_long) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_long), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_long[::interval]]

# Adjust the overlapping xticks and yticks by setting the first ytick to be empty
plt.yticks(ticks=[0] + tick_values[1:], labels=[""] + tick_labels[1:], fontsize=14, rotation=90)
# First scatter plot for the edges (larger size, different color)
scatter = plt.scatter(data_points[:, 0], data_points[:, 1], c='purple', s=40, edgecolors='white', linewidths=2, label='Data Collection Points Edges')


ax.set_xlim([0, map_.shape[1]])
ax.set_ylim([0, map_.shape[0]])
ax.set_aspect('equal')

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
from tqdm import tqdm
import matplotlib

# Set font style
matplotlib.rcParams['font.family'] = 'Times New Roman'

# Function to convert linear power to dB
def lin_to_dB(pow_lin):
    return 10 * np.log10(np.array(pow_lin))

# Function to aggregate data into larger blocks by averaging
def aggregate_data(data, block_size):
    shape = (data.shape[0] // block_size, data.shape[1] // block_size)
    aggregated_data = np.zeros(shape)

    for i in range(shape[0]):
        for j in range(shape[1]):
            aggregated_data[i, j] = np.mean(data[i*block_size:(i+1)*block_size, 
                                                 j*block_size:(j+1)*block_size])
    return aggregated_data

# Function to draw aggregated squares based on power and duty cycle data
def draw_aggregated_squares(ax, power_data, duty_cycle_data, block_size):
    power_cmap = plt.cm.jet
    power_norm = mcolors.Normalize(vmin=np.min(power_data), vmax=np.max(power_data))
    pixel_size = block_size

    for i in tqdm(range(power_data.shape[0])):
        for j in range(power_data.shape[1]):
            square_side_length = (duty_cycle_data[i, j])**(1/1.1) * pixel_size
            square_color = power_cmap(power_norm(power_data[i, j]))
            square_x = j * pixel_size + (pixel_size - square_side_length) / 2
            square_y = i * pixel_size + (pixel_size - square_side_length) / 2
            # Inside the draw_aggregated_squares function
            ax.add_patch(plt.Rectangle((square_x, square_y), square_side_length, square_side_length, 
                                       color=square_color, alpha=0.5)) # Add transparency


import os
import scipy.io as sio
import matplotlib.pyplot as plt
map_folderdir = "./"
directory = os.listdir(map_folderdir)
flag = 0
for fname in directory:
    if "SLCmap" in fname:
        map_file = os.path.join(map_folderdir, fname)
        flag = 1

if flag == 0:
    errorMessage = 'Error: The file does not exist in the folder:\n ' + map_folderdir
    warnings.warn(errorMessage)

print('Now reading ' + map_file + "\n")
x = sio.loadmat(map_file)
map_struct = x['SLC']

# Define a new struct named SLC
SLC = map_struct[0][0]
column_map = dict(zip([name for name in SLC.dtype.names], [i for i in range(len(SLC.dtype.names))]))

map_ = SLC[column_map["dem"]] + 0.3048 * SLC[column_map["hybrid_bldg"]]
map_ = map_[:5200, :5800]
x = np.linspace(0, map_.shape[1]//10, map_.shape[1]//10 , endpoint=False)
y = np.linspace(0, map_.shape[0]//10, map_.shape[0]//10, endpoint=False)
X, Y = np.meshgrid(x, y)
# After defining the X and Y grids
print(f"X grid shape: {X.shape}")
print(f"Y grid shape: {Y.shape}")
print(f"Power data shape: {aggregated_power.shape}")
print(f"Building data shape: {aggregated_bldg.shape}")

# Check the first and last few coordinates to see if they align
print(f"X grid start and end: {X[0, 0]}, {X[-1, -1]}")
print(f"Y grid start and end: {Y[0, 0]}, {Y[-1, -1]}")
print(f"Power data start and end values: {aggregated_power[0, 0]}, {aggregated_power[-1, -1]}")
print(f"Building data start and end values: {aggregated_bldg[0, 0]}, {aggregated_bldg[-1, -1]}")

dem = SLC[column_map["dem"]]
dem = dem[:5200, :5800]
bldg = 0.3048 * SLC[column_map["hybrid_bldg"]]
bldg = bldg[:5200, :5800]

#map_ = SLC[column_map["dem"]] + 0.3048 * SLC[column_map["hybrid_bldg"]]

def replace_border_zeros(arr):
    rows, cols = arr.shape

    # Function to find nearest non-zero for a given position
    def nearest_non_zero(i, j):
        for d in range(1, max(rows, cols)):
            for di in range(-d, d + 1):
                for dj in range(-d, d + 1):
                    if 0 <= i + di < rows and 0 <= j + dj < cols and arr[i + di, j + dj] != 0:
                        return arr[i + di, j + dj]
        return 0  # In case there are no non-zero neighbors

    # Replace zeros on top and bottom rows
    for j in range(cols):
        if arr[0, j] == 0:
            arr[0, j] = nearest_non_zero(0, j)
        if arr[rows - 1, j] == 0:
            arr[rows - 1, j] = nearest_non_zero(rows - 1, j)

    # Replace zeros on left and right columns
    for i in range(rows):
        if arr[i, 0] == 0:
            arr[i, 0] = nearest_non_zero(i, 0)
        if arr[i, cols - 1] == 0:
            arr[i, cols - 1] = nearest_non_zero(i, cols - 1)

    return arr

# Example array

# Replace border zeros
dem = replace_border_zeros(dem)
map_ = map_[::10, ::10]
dem = dem[::10, ::10]
bldg = bldg[::10, ::10]
power_data = lin_to_dB(np.load("var_pred_3610_3650_2.npy")).reshape((521, 585))[:520, :580]
power_data[np.isnan(power_data)]=0
print(power_data.shape)
duty_cycle_data = np.load("confidence_level.npy")
duty_cycle_data = duty_cycle_data[:520, :580]

block_size=10
# Initialize the plot
fig, ax = plt.subplots(figsize=(13, 10))

# Aggregate the data
aggregated_power = aggregate_data(power_data, block_size)
aggregated_duty_cycle = aggregate_data(duty_cycle_data, block_size)
aggregated_dem = aggregate_data(dem, block_size)
aggregated_bldg = aggregate_data(bldg, block_size)
aggregated_X = aggregate_data(X, block_size)
aggregated_Y = aggregate_data(Y, block_size)

plt.contourf(X, Y, bldg, 100, cmap='gist_gray', alpha=0.5) # Adjust alpha for transparency

# Draw the aggregated squares (spectrum occupancy)
draw_aggregated_squares(ax, aggregated_power, aggregated_duty_cycle, block_size)

# Create colorbar for average power
sm = plt.cm.ScalarMappable(cmap=plt.cm.jet, norm=plt.Normalize(vmin=np.min(power_data), vmax=np.max(power_data)))
cbar_power = plt.colorbar(sm, ax=ax, shrink=1, pad=0.02)
cbar_power.ax.tick_params(labelsize=19)
cbar_power.set_label('Signal Variance [dBX^2]', size=20)  # adjust the size value as needed

# Set axis labels and titles
ax.set_xlabel('UTM$_E$ [m]', fontsize=24)
ax.set_ylabel('UTM$_N$ [m]', fontsize=24)
plt.title("3610-3650 MHz Variation & Confidence", fontsize=27)

N1 = map_.shape[0]
N2 = map_.shape[1]

en = SLC[column_map["axis"]]
UTM_long = np.linspace(en[0, 2], en[0, 3] - SLC[column_map["cellsize"]], N1)
UTM_lat = np.linspace(en[0, 0], en[0, 1] - SLC[column_map["cellsize"]], N2)
UTM_long = np.squeeze(UTM_long)
UTM_lat = np.squeeze(UTM_lat)
interval = 100
ax.set_xticks(range(0, len(UTM_lat), interval))
ax.set_xticklabels(UTM_lat[::interval], fontsize=14, rotation=0)

ax.set_yticks(range(0, len(UTM_long), interval))
ax.set_yticklabels(UTM_long[::interval], fontsize=14)



interval = max(1, len(UTM_lat) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_lat), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_lat[::interval]]

# Set the tick values and labels
plt.xticks(ticks=tick_values, labels=tick_labels, fontsize=14, rotation=0)

interval = max(1, len(UTM_long) // 5)  # interval for ticks to avoid overlap, you can adjust if needed

# Assuming 'UTM_lat' is your data and 'interval' is defined
tick_values = list(range(0, len(UTM_long), interval))
tick_labels = ['{:.1f}'.format(lat) for lat in UTM_long[::interval]]

# Adjust the overlapping xticks and yticks by setting the first ytick to be empty
plt.yticks(ticks=[0] + tick_values[1:], labels=[""] + tick_labels[1:], fontsize=14, rotation=90)
# First scatter plot for the edges (larger size, different color)
scatter = plt.scatter(data_points[:, 0], data_points[:, 1], c='purple', s=40, edgecolors='white', linewidths=2, label='Data Collection Points Edges')


ax.set_xlim([0, map_.shape[1]])
ax.set_ylim([0, map_.shape[0]])
ax.set_aspect('equal')

# Show the plot
plt.tight_layout()
plt.show()
